In [1]:
import pandas as pd

df = pd.read_csv('./datasets/adult.csv',delimiter=',', skipinitialspace=True)
del df['fnlwgt']
del df['education-num']

for col in df.columns:
    if '?' in df[col].unique():
        df[col][df[col] == '?'] = df[col].value_counts().index[0]

<ipython-input-1-16d482bc9b74>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if '?' in df[col].unique():
<ipython-input-1-16d482bc9b74>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][df[col] == '?'] = df[col].value_counts().index[0]


In [2]:
columns = df.columns.tolist()
columns = columns[-1:] + columns[:-1]
df = df[columns]
class_name = 'class'
possible_outcomes = list(df[class_name].unique())

In [3]:
df.head(5)

,class,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,<=50K,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,<=50K,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,<=50K,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,<=50K,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,<=50K,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [4]:
possible_outcomes

['<=50K', '>50K']

In [5]:
integer_features = list(df.select_dtypes(include=['int64']).columns)
float_features = list(df.select_dtypes(include=['float64']).columns)
string_features = list(df.select_dtypes(include=['object']).columns)
type_features = {
    'integer': integer_features,
    'float': float_features,
    'string': string_features,
}

In [6]:
type_features

{'integer': ['age', 'capital-gain', 'capital-loss', 'hours-per-week'],
 'float': [],
 'string': ['class',
  'workclass',
  'education',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'sex',
  'native-country']}

In [9]:
### Seperate to numerical categorical
numerical_cols = type_features['integer'] + type_features['float']
categorical_cols = type_features['string']

In [11]:
numerical_cols

['age', 'capital-gain', 'capital-loss', 'hours-per-week']

In [12]:
categorical_cols

['class',
 'workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [13]:
from sklearn.preprocessing import LabelEncoder

def label_encode(df, columns, encoder_dict=None):
    df_temp = df.copy(deep=True)

    if encoder_dict:
        for col in columns:
            col_encoder = encoder_dict[col]
            df_temp[col] = col_encoder.transform(df_temp[col])
    else:
        encoder_dict = {}
        for col in columns:
            encoder = LabelEncoder()
            df_temp[col] = encoder.fit_transform(df_temp[col])
            encoder_dict[col] = encoder

    return df_temp, encoder_dict

In [17]:
def label_decode(df, columns, encoder_dict):
    temp_df = df.copy(deep=True)
    for col in columns:
        encoder = encoder_dict[col]
        temp_df[col] = encoder.inverse_transform(temp_df[col])
    return temp_df

In [14]:
df, encode_dict = label_encode(df, categorical_cols)

In [15]:
df

,class,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0,39,6,9,4,0,1,4,1,2174,0,40,38
1,0,50,5,9,2,3,0,4,1,0,0,13,38
2,0,38,3,11,0,5,1,4,1,0,0,40,38
3,0,53,3,1,2,5,0,2,1,0,0,40,38
4,0,28,3,9,2,9,5,2,0,0,0,40,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0,27,3,7,2,12,5,4,0,0,0,38,38
32557,1,40,3,11,2,6,0,4,1,0,0,40,38
32558,0,58,3,11,6,0,4,4,0,0,0,40,38
32559,0,22,3,11,4,0,3,4,1,0,0,20,38


In [16]:
categorical_cols

['class',
 'workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [22]:
pd.get_dummies(df['workclass'], drop_first=True)

,1,2,3,4,5,6,7
0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
32556,0,0,1,0,0,0,0
32557,0,0,1,0,0,0,0
32558,0,0,1,0,0,0,0
32559,0,0,1,0,0,0,0


In [24]:
def transform_to_dummy(df, columns):
    for col in columns:
        df = pd.concat([df,pd.get_dummies(df[col], prefix=col)],axis=1)
        df.drop([col],axis=1, inplace=True)
    return df

In [25]:
df = transform_to_dummy(df, categorical_cols)

In [26]:
df.head(5)

,age,capital-gain,capital-loss,hours-per-week,class_0,class_1,workclass_0,workclass_1,workclass_2,workclass_3,...,native-country_31,native-country_32,native-country_33,native-country_34,native-country_35,native-country_36,native-country_37,native-country_38,native-country_39,native-country_40
0,39,2174,0,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,0,0,13,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,0,0,40,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,53,0,0,40,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,28,0,0,40,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [48]:
from sklearn.tree import DecisionTreeClassifier